# This is a notebook for querying the EMODnet Chemistry instance.

- You can run each cell individually by pressing "shift + enter".
- Please run the first few cells in order for the input widgets to appear.
- For more information, questions, bugs, please contact us on Slack:
  - https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2n95bd4rk-XMtZ9AVhVapwrFna1J5qmA.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token that you should receive via Blue-Cloud between the '' " in the cell below.


In [1]:
Token = ""

#### Importing the required packages.


In [2]:
import requests
import json
import xarray as xr
import datetime
from ipywidgets import interact, widgets, interactive
from IPython.display import Javascript, display
import pandas as pd
import os

#### This retrieves the available parameters from the footprint.


In [16]:
response = requests.get(
    "https://beacon-emod-chem.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
data = response.json()
pd.set_option("display.max_rows", 500)
attributes = pd.DataFrame(data)
params = attributes.to_numpy().flatten()
params

array(['Access restriction', 'Access restriction.comments',
       'Access restriction.long_name', 'Access/ordering of data',
       'Access/ordering of data.comments',
       'Access/ordering of data.long_name', 'Alternative cruise name',
       'Alternative cruise name.comments',
       'Alternative cruise name.long_name', 'Alternative station name',
       'Alternative station name.comments',
       'Alternative station name.long_name', 'Bot. Depth',
       'Bot. Depth.comments', 'Bot. Depth.long_name', 'Bot. Depth.units',
       'Bottle', 'Bottle.comments', 'Bottle.long_name', 'Bottle.units',
       'Bottle_qc', 'C22_flag', 'C22_flag.comments', 'C22_flag.long_name',
       'C22_flag.units', 'C22_flag_qc', 'CDI-partner',
       'CDI-partner.comments', 'CDI-partner.long_name',
       'CDI-record creation date', 'CDI-record creation date.comments',
       'CDI-record creation date.long_name', 'CDI-record id',
       'CDI-record id.comments', 'CDI-record id.long_name',
       'CSR plat

#### In the widgets below, you should select your preferred filters and press "Execute all cells" to run the whole notebook.


In [17]:
def f(parameter, minlon, maxlon, minlat, maxlat, mindepth, maxdepth, mintemp, maxtemp):
    return (
        parameter,
        minlon,
        maxlon,
        minlat,
        maxlat,
        mindepth,
        maxdepth,
        mintemp,
        maxtemp,
    )


w = interactive(
    f,
    parameter=params,
    minlon=(-180, 180, 1),
    maxlon=(-180, 180, 1),
    minlat=(-90, 90, 1),
    maxlat=(-90, 90, 1),
    mindepth=widgets.IntSlider(min=0, max=5000, value=0),
    maxdepth=widgets.IntSlider(min=0, max=5000, value=10),
    mintemp=widgets.DatePicker(
        description="Start date:", value=datetime.date(2000, 1, 1)
    ),
    maxtemp=widgets.DatePicker(
        description="End date:", value=datetime.date(2024, 1, 1)
    ),
)

display(w)


def run_all(ev):
    display(
        Javascript(
            "IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())"
        )
    )


button = widgets.Button(description="Execute all cells")
button.on_click(run_all)
display(button)

interactive(children=(Dropdown(description='parameter', options=('Access restriction', 'Access restriction.com…

Button(description='Execute all cells', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Storing the chosen filters.


In [38]:
parameter = w.kwargs["parameter"]
mintemp = w.kwargs["mintemp"]
maxtemp = w.kwargs["maxtemp"]
minlon = w.kwargs["minlon"]
maxlon = w.kwargs["maxlon"]
minlat = w.kwargs["minlat"]
maxlat = w.kwargs["maxlat"]
mindepth = w.kwargs["mindepth"]
maxdepth = w.kwargs["maxdepth"]

#### The fields below are fixed for the EMODnet Chemistry instance.


In [39]:
dateref = datetime.date(1921, 1, 1)  # Days since 1921
mintemporal = mintemp.strftime("%Y-%m-%dT%H:%M:%S")
maxtemporal = maxtemp.strftime("%Y-%m-%dT%H:%M:%S")
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

#### Query body based on your input fields, you can add other "query_parameters" and "filters" to suit your needs.

- For more query examples, you can take a look at https://maris-development.github.io/beacon/available-nodes/blue-cloud-2026/emodnet-chemistry.html.
- In this example, you can see that I added a filter for the parameter, i.e. Temperature measurements should be between -2 and 40 degrees Celsius. Of course, you can change the values based on your parameter, or remove it entirely.


In [40]:
def query(
    parameter,
    mintemporal,
    maxtemporal,
    mindepth,
    maxdepth,
    minlon,
    maxlon,
    minlat,
    maxlat,
):
    body = {
        "query_parameters": [
            {"column_name": parameter, "alias": parameter},
            {"column_name": "yyyy-mm-ddThh:mm:ss.sss", "alias": "TIME"},
            {"column_name": "Depth", "alias": "DEPTH"},
            {"column_name": "Longitude", "alias": "LONGITUDE"},
            {"column_name": "Latitude", "alias": "LATITUDE"},
        ],
        "filters": [
            {
                "for_query_parameter": "TIME",
                "min": mintemporal,
                "max": maxtemporal,
                "cast": "timestamp",
            },
            {"for_query_parameter": "DEPTH", "min": mindepth, "max": maxdepth},
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
            {"for_query_parameter": parameter, "min": -2, "max": 40},
        ],
        "output": {"format": "netcdf"},
    }
    return body


query = query(
    parameter,
    mintemporal,
    maxtemporal,
    mindepth,
    maxdepth,
    minlon,
    maxlon,
    minlat,
    maxlat,
)

#### This is the post request that is sent to Beacon with the above specified body.


In [41]:
response = requests.post(
    "https://beacon-emod-chem.maris.nl/api/query",
    json.dumps(query),
    headers={"Authorization": f"Bearer {Token}",
             "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

#### This will create a Netcdf file in your directory with the name based on your filters, the output is shown here in a dataframe.


In [42]:
if not os.path.exists("./DataOutput"):
    os.makedirs("DataOutput")

open(
    f'./DataOutput/EMODNET_CHEM_{parameter}_{regionname}_{mintemp.strftime("%Y-%m-%d")}-{
        maxtemp.strftime("%Y-%m-%d")}_[{mindepth}-{maxdepth}m].nc',
    "wb",
).write(response.content)
df = xr.open_dataset(
    f'./DataOutput/EMODNET_CHEM_{parameter}_{regionname}_{mintemp.strftime(
        "%Y-%m-%d")}-{maxtemp.strftime("%Y-%m-%d")}_[{mindepth}-{maxdepth}m].nc'
).to_dataframe()
df

,ITS-90 water temperature,TIME,DEPTH,LONGITUDE,LATITUDE,dataset_id
INSTANCE,,,,,,
0,10.590,2001-02-24 09:07:00,1.983256,-4.498993,47.750000,23300.0
1,10.590,2001-02-24 09:07:00,2.974876,-4.498993,47.750000,23300.0
2,10.580,2001-02-24 09:07:00,3.966492,-4.498993,47.750000,23300.0
3,10.580,2001-02-24 09:07:00,4.958103,-4.498993,47.750000,23300.0
4,10.580,2001-02-24 09:07:00,5.949709,-4.498993,47.750000,23300.0
...,...,...,...,...,...,...
36406,12.291,2000-05-01 08:11:00,7.933345,-2.803833,47.150333,24076.0
36407,12.299,2000-05-01 08:11:00,8.924992,-2.803833,47.150333,24076.0
36408,12.298,2000-05-01 08:11:00,9.916634,-2.803833,47.150333,24076.0
